# Reason for this test #

### Documentation of an idea ###

The point of adding the last fcost constraint expression, which is trivial anyways because it comes from <b>1</b> - <b>2</b> - <b>3</b>, is that these contraints then can be changed easily when changing the destination node. Just change the RHS of the constraint equations, that is subtract 1 or 0 accordingly.

In [1]:
from pyqubo import Array,Constraint,Placeholder

In [2]:
edges = [(1,2),(1,3),(2,3),(2,4),(3,4)]
weights = [5,8,2,7,4]
nodes = 4
x = Array.create(name = 'x', shape =len(edges), vartype = 'BINARY') 

Add the Linear Terms to the constraints

In [3]:
i = 0
fcost = 0
for i in range(len(edges)):
    fcost += (weights[i]*x[i]) ## Put the expression for constraints in

Add the Quadratic Terms to the constraints

In [4]:
# Try to algorithmically add the constraints in
## Set the source and destination nodes ##
source = 1
destination = 4
p = 27

node=1
while(node<=nodes):
    sum = 0
    for j in range(len(edges)):
        if edges[j][0] == node: ##Outgoing from the current node
            sum += x[j]
        elif edges[j][1] == node: ##Incoming into the current node
            sum -= x[j]
    
    if node == source:
        sum -= 1
    elif node== destination:
        sum += 1
        
    fcost += p*((sum)**2)
        
    # print(sum, "For Node "+str(node))
    node+=1


Now Manually add in these constraints to fcost. The reason we're not doing it algorithmically is due to the nature of the QUBO model variables. For some reason only when each of the terms are added in one line does it show the correct results. Make sure to not add any brackets when hard-coding the expressions.

In [5]:
print(fcost)

((27.000000 * ((1.000000 + (-1.000000 * Binary('x[4]')) + 0.000000 + (-1.000000 * Binary('x[3]'))) * (1.000000 + (-1.000000 * Binary('x[4]')) + 0.000000 + (-1.000000 * Binary('x[3]'))))) + (27.000000 * ((Binary('x[4]') + (-1.000000 * Binary('x[2]')) + 0.000000 + (-1.000000 * Binary('x[1]'))) * (Binary('x[4]') + (-1.000000 * Binary('x[2]')) + 0.000000 + (-1.000000 * Binary('x[1]'))))) + (27.000000 * ((Binary('x[3]') + Binary('x[2]') + 0.000000 + (-1.000000 * Binary('x[0]'))) * (Binary('x[3]') + Binary('x[2]') + 0.000000 + (-1.000000 * Binary('x[0]'))))) + (27.000000 * ((-1.000000 + Binary('x[1]') + 0.000000 + Binary('x[0]')) * (-1.000000 + Binary('x[1]') + 0.000000 + Binary('x[0]')))) + (4.000000 * Binary('x[4]')) + (7.000000 * Binary('x[3]')) + (2.000000 * Binary('x[2]')) + (8.000000 * Binary('x[1]')) + 0.000000 + (5.000000 * Binary('x[0]')))


In [6]:
model = fcost.compile()

In [7]:
print(model.to_qubo())

({('x[4]', 'x[1]'): -54.0, ('x[0]', 'x[0]'): 5.0, ('x[2]', 'x[2]'): 56.0, ('x[4]', 'x[2]'): -54.0, ('x[3]', 'x[2]'): 54.0, ('x[3]', 'x[3]'): 7.0, ('x[1]', 'x[0]'): 54.0, ('x[2]', 'x[1]'): 54.0, ('x[4]', 'x[3]'): 54.0, ('x[3]', 'x[0]'): -54.0, ('x[2]', 'x[0]'): -54.0, ('x[4]', 'x[4]'): 4.0, ('x[1]', 'x[1]'): 8.0}, 54.0)


In [8]:
linear, quadratic, offset = model.to_ising()
print("Linear Coefficients", linear)
print("Quadratic Coefficients", quadratic)

## The objective function is then made from these linear and quadratic terms ##
## The objective function represents the Problem Hamiltonian Hp ##

Linear Coefficients {'x[0]': -11.0, 'x[1]': 17.5, 'x[2]': 28.0, 'x[3]': 17.0, 'x[4]': -11.5}
Quadratic Coefficients {('x[0]', 'x[1]'): 13.5, ('x[0]', 'x[2]'): -13.5, ('x[2]', 'x[3]'): 13.5, ('x[0]', 'x[3]'): -13.5, ('x[2]', 'x[4]'): -13.5, ('x[3]', 'x[4]'): 13.5, ('x[1]', 'x[2]'): 13.5, ('x[1]', 'x[4]'): -13.5}


In [9]:
# general imports
import numpy as np
import matplotlib.pyplot as plt
# magic word for producing visualizations in notebook
%matplotlib inline
import string
import time

In [10]:
# AWS imports: Import Braket SDK modules
from braket.circuits import Circuit, Gate, Observable
from braket.devices import LocalSimulator
from braket.aws import AwsDevice, AwsQuantumTask

In [12]:
# the IonQ device
device = AwsDevice("arn:aws:braket:::device/qpu/ionq/ionQdevice")
supported_gates = device.properties.action['braket.ir.jaqcd.program'].supportedOperations
# print the supported gate set
print('Gate set supported by the IonQ device:\n', supported_gates)

Gate set supported by the IonQ device:
 ['x', 'y', 'z', 'rx', 'ry', 'rz', 'h', 'cnot', 's', 'si', 't', 'ti', 'v', 'vi', 'xx', 'yy', 'zz', 'swap']


In [13]:
def create_circuit(beta, gamma):
    ## initializing the initial qubit state with H gates ##
    circuit = Circuit()
    n_qubits = len(edges)

    for qubit in range(n_qubits):
        circuit.h(qubit)
    
    ## Implementing the problem Hamiltonian ##
    for qubit in range(n_qubits):
        linear_coeff = linear.get('x['+str(qubit)+']')
        circuit = circuit.rz(qubit, -1*linear_coeff)

    #Algorithmic method to add the ZZ gates - CHECK TO SEE IF IT AFFECTS THE RESULTS(it should'nt because they commute)
    for i in range(len(quadratic)):
        qubit_1 = int(list(quadratic.keys())[i][0][2])
        qubit_2 = int(list(quadratic.keys())[i][1][2])

        quadratic_coeff = quadratic.get(('x['+str(qubit_1)+']', 'x['+str(qubit_2)+']'))

        circuit.zz(qubit_1, qubit_2, quadratic_coeff*gamma)
    
    ## Implementing the Mixer Hamiltonian ##
    for qubit in range(n_qubits):
        circuit.rx(qubit, 2*beta) # theta=2*beta because rx rotates the qubit about X by theta/2 angle
        
    return circuit

* <b>Remember that there are 3 parameters that can be varied - beta, gamma and penalty p </b>

In [14]:
## Expectation value of the Hamiltonian is basically the expected cost value which we can get from an average of the
## cost values over all states that have occurred ##
def compute_expectation(counts, shots):
    
    expectation = 0
    sum = 0
    states = list(counts.keys())
    for i in range(len(states)):
        state = states[i] # string variable of the current qubit states
        state_cost = 0
        for j in range(len(state)): # Convention of the states is that the left-most qubit is the first qubit q0
            state_cost = state_cost + int(state[j])*weights[j]
        
        expectation = expectation + state_cost*counts.get(state)
        
    expectation /= 1000
    # print(expectation)
    return expectation

In [17]:
## Now we measure the circuit ##
def expectation_execute_circuit(param):
    ## Set up the device to run the circuit
    # set up device
    ionq = AwsDevice("arn:aws:braket:::device/qpu/ionq/ionQdevice")
    
    ## QAOA parameters to be optimized such that the eigenvalue Cost(β, γ) can be minimized ##
    beta = param[0]
    gamma = param[1]
    
    circuit = create_circuit(beta, gamma)
    
    shots = 1000
    result = ionq.run(circuit, shots).result()
    counts = result.measurement_counts
    
    print()
    
    return compute_expectation(counts, shots)    

In [ ]:
circuit = create_circuit(0.91655593, 0.93386801)

device = AwsDevice("arn:aws:braket:::device/qpu/ionq/ionQdevice")
result = device.run(circuit, shots = 1000).result()
counts = result.measurement_counts

print(counts)

In [18]:
## Classical Optimizer ##

from scipy.optimize import minimize

res = minimize(expectation_execute_circuit,
               [1.0, 1.0],
               method='COBYLA')
print(res)

TypeError: object of type 'int' has no len()

## Analyzing the Results

In [16]:
beta = res.get('x')[0]
gamma = res.get('x')[1]
circuit = create_circuit(0.28517317, -5.05969577)

device = LocalSimulator()
result = device.run(circuit, shots = 1000).result()
counts = result.measurement_counts

print(counts)

# plot using Counter
#plt.bar(counts.keys(), counts.values())
#plt.xlabel('bitstrings')
#plt.ylabel('counts')

Counter({'10101': 123, '11100': 96, '00111': 71, '01001': 67, '10001': 65, '10010': 64, '00000': 52, '11011': 50, '10011': 34, '01000': 34, '10100': 27, '00101': 27, '01101': 25, '11010': 25, '00001': 25, '10111': 24, '01011': 22, '00010': 21, '00100': 21, '01110': 20, '11111': 19, '10110': 15, '11001': 14, '11101': 14, '11110': 12, '01100': 9, '00011': 6, '01010': 5, '11000': 5, '01111': 5, '10000': 3})


## Post-Processing

Remove the output states that are not possible (among the top ten most probable states) and then check for the most probable states.
Check for joined paths, that is check that if a path enum ending with a number exists, then another path enum starting with the same number also exists, unless its a source or destination. They should always be 1.

In [17]:
def check_state(s):
    
    ## Firstly check if the path starts from a source and ends at a destination
    source_flag = False
    destination_flag = False
    multiple_branches = False
    continuity_flag = True
    
    starting_nodes = []
    ending_nodes = []
    
    ## Check to see if the source and destination nodes exist in the network
    i=0
    for i in range(len(s)):
        if(s[i] == '1'):
            
            if(edges[i][0] == source):
                source_flag = True
            if(edges[i][1] == destination):
                destination_flag = True
                
            starting_nodes += [edges[i][0]]
            ending_nodes += [edges[i][1]]
            
    ## Now check if a node repeats itself in starting or ending_nodes. If yes, set multiple_branches
    i = 0
    for i in range(len(starting_nodes)):
        cnt1 = starting_nodes.count(starting_nodes[i])
        cnt2 = ending_nodes.count(ending_nodes[i])
        if cnt1 > 1 or cnt2 > 1:
            multiple_branches = True
            break
            
    
    ## Then iteratively go through ending nodes and check if the same node exists in the next starting_nodes index                
    ## This is an easier check for continuity and necessarily requires the edges nodes to be in some order
    ## Also go with the thumb rule that the destination node will be the last value of ending_nodes
    for i in range(len(ending_nodes)-1):
        if starting_nodes[i+1] != ending_nodes[i]:
            continuity_flag = False
            break
                    
    if source_flag and destination_flag and continuity_flag and (not multiple_branches):
        return True
    else:
        return False

# s = '10010'
# flag = check_state(s)
# print(flag)

## After Post-processing ##

In [18]:
states = list(counts.keys())
possible_states = []
i = 0
for i in range(len(states)):
    s = states[i]
    flag = check_state(s)
    if flag:
        possible_states += [s]
    
del states

i = 0
for i in range(len(possible_states)):
    print(possible_states[i]+':'+str(counts[possible_states[i]]))

01001:67
10101:123
10010:64


Therefore the most probable possible state is 10101.